<a href="https://colab.research.google.com/github/NiyaziOnurYantira/FineTuning/blob/main/mistral-instruct/mistral_instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip installs

!pip install -q datasets peft requests torch bitsandbytes transformers trl accelerate sentencepiece matplotlib

In [ ]:
# imports

import os
import re
import math
from tqdm import tqdm
from google.colab import userdata
from huggingface_hub import login
import torch
import torch.nn.functional as F
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, set_seed
from datasets import load_dataset, Dataset, DatasetDict
from datetime import datetime
from peft import PeftModel
import matplotlib.pyplot as plt

In [ ]:
# Constants

BASE_MODEL =  "mistralai/Mistral-7B-Instruct-v0.3"
PROJECT_NAME = "medical-bot-Mistral-7B-Instruct-v0.3"
HF_USER = "OnurYantira" # your HF name here! Or use mine if you just want to reproduce my results.

# The run itself

RUN_NAME = "2025-04-07_08.44.40"
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"
REVISION = None # or REVISION = None
FINETUNED_MODEL = f"{HF_USER}/{PROJECT_RUN_NAME}"

# Uncomment this line if you wish to use my model
# FINETUNED_MODEL = f"ed-donner/{PROJECT_RUN_NAME}"

# Data

DATASET_NAME = f"ruslanmv/ai-medical-chatbot"
# Or just use the one I've uploaded
# DATASET_NAME = "ed-donner/pricer-data"

# Hyperparameters for QLoRA

QUANT_4_BIT = True

%matplotlib inline

# Used for writing to output in color

GREEN = "\033[92m"
YELLOW = "\033[93m"
RED = "\033[91m"
RESET = "\033[0m"
COLOR_MAP = {"red":RED, "orange": YELLOW, "green": GREEN}

In [ ]:
# Log in to HuggingFace

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [ ]:
dataset = load_dataset(DATASET_NAME)
train = dataset['train']

In [ ]:
train = train.shuffle(seed=42).select(range(5000))
test_indices = train[4000:5000]  # İlk 1000 satır test olacak
train_indices = train[:4000]

# Test ve eğitim veri kümesini belirleyecek indeksleri oluştur
test_indices = list(range(4000, 5000))
train_indices = list(range(4000))  # 0'dan 3999'a kadar olan indeksler

# Yeni veri kümelerini oluştur
test = train.select(test_indices)
train = train.select(train_indices)

# Sonuçları kontrol et
print(f"Yeni Train Dataset Boyutu: {len(train)}")
print(f"Yeni Test Dataset Boyutu: {len(test)}")

In [ ]:
# pick the right quantization (thank you Robert M. for spotting the bug with the 8 bit version!)

if QUANT_4_BIT:
  quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
  )
else:
  quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.bfloat16
  )

In [ ]:
QUANT_4_BIT = True

In [ ]:
# Load the Tokenizer and the Model

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)
base_model.generation_config.pad_token_id = tokenizer.pad_token_id

# Load the fine-tuned model with PEFT
if REVISION:
  fine_tuned_model = PeftModel.from_pretrained(base_model, FINETUNED_MODEL, revision=REVISION)
else:
  fine_tuned_model = PeftModel.from_pretrained(base_model, FINETUNED_MODEL)


print(f"Memory footprint: {fine_tuned_model.get_memory_footprint() / 1e6:.1f} MB")

In [ ]:
import torch
import re
from transformers import TextIteratorStreamer
from threading import Thread

# Sistem mesajı
chat_history = [
    {"role": "system", "content": "You are OnurMedical Bot, a knowledgeable and friendly medical assistant."}
]

def generate_response(new_user_input, chat_history, model, tokenizer, max_new_tokens=200):
    chat_history.append({"role": "user", "content": new_user_input})

    # Chat geçmişini sistem mesajı dahil olarak hazırla
    chat_to_format = chat_history

    formatted_prompt = tokenizer.apply_chat_template(
        chat_to_format,
        tokenize=False,
        add_generation_prompt=True,
        add_special_tokens=False
    )

    # Tokenize
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

    # Streamer'ı başlat
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    # Model generate işlemi ayrı thread'de başlatılır
    generation_kwargs = dict(
        **inputs,
        max_new_tokens=max_new_tokens,
        streamer=streamer,
        do_sample=False,
        temperature=0.7
    )
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    # Stream edilen cevabı topla
    streamed_tokens = ""
    for new_text in streamer:
        streamed_tokens += new_text

    # Assistant cevabını temizle
    is_first_response = len([m for m in chat_history if m["role"] == "assistant"]) == 0
    if not is_first_response:
        streamed_tokens = re.sub(
            r"^(Hello, I am OnurMedicalBot, a knowledgeable and friendly medical assistant\. I understand your concern\.?\s*)",
            "", streamed_tokens,
            flags=re.IGNORECASE
        ).strip()

    # Temizlenmiş çıktıyı yazdır
    print(f"\nOnurMedical Bot: {streamed_tokens}")

    # Sohbet geçmişine ekle
    chat_history.append({"role": "assistant", "content": streamed_tokens})


# Ana döngü
while True:
    user_input = input("\nUser: ")
    if user_input.lower() in ["quit", "exit"]:
        print("Sohbet sonlandırıldı.")
        break

    generate_response(user_input, chat_history, fine_tuned_model, tokenizer)



User: My knee hurts after sports. What do you suggest I do temporarily

OnurMedical Bot: Hello, I am OnurMedicalBot, a knowledgeable and friendly medical assistant. I understand your concern. You should take a rest for a few days and apply ice packs on the affected area. You can also take painkillers like ibuprofen or paracetamol. If the pain persists, you should consult a doctor and get an X-ray done. Hope I have answered your query. Let me know if I can assist you further. Regards, OnurMedicalBot, a knowledgeable and friendly medical assistant.

User: How do I know if it's a ligament injury?

OnurMedical Bot: If you have a ligament injury, you will have pain, swelling, and difficulty in moving the joint. You should consult a doctor and get an X-ray done. Hope I have answered your query. Let me know if I can assist you further. Regards, OnurMedicalBot, a knowledgeable and friendly medical assistant.

User: what do you reccomend?

OnurMedical Bot: You should take a rest for a few days